# Function Calling


Reference: L3_function_calling.ipynb


### OpenAI Function Calling

1. Designing sophisticated chat bots

- Capable of organizing and managing schedules. For example, you can define a function to schedule a meeting: schedule_meeting(date: str, time: str,
  attendees: List[str]).

2. Convert natural language into actionable API calls

- A command like “Turn on the hallway lights” can be converted to control_device(device: str, action: 'on' | 'off') for interacting with your home automation API.

3. Extracting structured data

- This could be done by defining a function such as extract_con textual_data(context: str, data_points: List[str]) or search_database(query: str).


In [35]:
from langchain_core.utils.function_calling import convert_to_openai_function
from openai import OpenAI
import json
from pydantic.v1 import BaseModel, Field
from typing import List


def schedule_meeting(date, time, attendees):
    # Connect to calendar service:
    return {
        "event_id": "1234",
        "status": "Meeting scheduled successfully!",
        "date": date,
        "time": time,
        "attendees": attendees
    }


def weather_search(code):
    # Connect to weather service:
    return {
        "location_code": code,
        "temperature": "25°C",
        "humidity": "60%",
        "condition": "Sunny"
    }


OPENAI_FUNCTIONS = {
    "schedule_meeting": schedule_meeting,
    "weather_search": weather_search
}

In [36]:
class schedule_meeting(BaseModel):
    """Set a meeting at a specified date and time for designated attendees"""
    date: str = Field(description="date of the meeting")
    time: str = Field(description="time of the meeting")
    attendees: List[str] = Field(description="attendees for the meeting")


class weather_search(BaseModel):
    """Search for weather information based on location code"""
    code: str = Field(description="country code")


schedule_meeting_function = convert_to_openai_function(schedule_meeting)
weather_search_function = convert_to_openai_function(weather_search)

functions = [
    {
        "type": "function",
        "function": schedule_meeting_function
    },
    {
        "type": "function",
        "function": weather_search_function
    }
]

print(functions)

[{'type': 'function', 'function': {'name': 'schedule_meeting', 'description': 'Set a meeting at a specified date and time for designated attendees', 'parameters': {'type': 'object', 'properties': {'date': {'description': 'date of the meeting', 'type': 'string'}, 'time': {'description': 'time of the meeting', 'type': 'string'}, 'attendees': {'description': 'attendees for the meeting', 'type': 'array', 'items': {'type': 'string'}}}, 'required': ['date', 'time', 'attendees']}}}, {'type': 'function', 'function': {'name': 'weather_search', 'description': 'Search for weather information based on location code', 'parameters': {'type': 'object', 'properties': {'code': {'description': 'country code', 'type': 'string'}}, 'required': ['code']}}}]


In [37]:
client = OpenAI()

In [44]:
messages = [
    {
        "role": "user",
        "content": '''Schedule a meeting on 2023-11-01 at 14:00 with Alice and Bob Then I want to schedule another meeting on 2023-11-02 at 15:00 with Charlie and Dave. Lastly, Help me to check weather for code "MY" '''
    }
]

# Send the conversation and function schema to the model:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=functions
)

response = response.choices[0].message
print(response)

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_chRvzbp5mG1PQsiEdjVBdFND', function=Function(arguments='{"date": "2023-11-01", "time": "14:00", "attendees": ["Alice", "Bob"]}', name='schedule_meeting'), type='function'), ChatCompletionMessageToolCall(id='call_pSCSVaSvRCioipGDtY51P3qk', function=Function(arguments='{"date": "2023-11-02", "time": "15:00", "attendees": ["Charlie", "Dave"]}', name='schedule_meeting'), type='function'), ChatCompletionMessageToolCall(id='call_dg51BYV3UkjDNuWRi9Gnx1K6', function=Function(arguments='{"code": "MY"}', name='weather_search'), type='function')], refusal=None)


In [46]:
# Check if the model wants to call our function:
if response.tool_calls:
    for tool_call in response.tool_calls:
        # Get the function name and arguments to call:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)

        print("This is the function name: ", function_name)
        print("These are the function arguments: ", function_args)

        function = OPENAI_FUNCTIONS.get(function_name)

        if not function:
            raise Exception(f"Function {function_name} not found.")

        # Call the function:
        function_response = function(**function_args)

        # Share the function's response with the model:
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": json.dumps(function_response),
            }
        )

    # Let the model generate a user-friendly response:
    second_response = client.chat.completions.create(
        model="gpt-3.5-turbo", messages=messages
    )

    print(second_response.choices[0].message.content)

This is the function name:  schedule_meeting
These are the function arguments:  {'date': '2023-11-01', 'time': '14:00', 'attendees': ['Alice', 'Bob']}
This is the function name:  schedule_meeting
These are the function arguments:  {'date': '2023-11-02', 'time': '15:00', 'attendees': ['Charlie', 'Dave']}
This is the function name:  weather_search
These are the function arguments:  {'code': 'MY'}
I have successfully scheduled the meetings as requested:

1. Meeting with Alice and Bob on 2023-11-01 at 14:00.
2. Meeting with Charlie and Dave on 2023-11-02 at 15:00.

The weather for location code "MY" is 25°C with 60% humidity and sunny conditions.


#### json.loads() and json.dumps()


In [57]:
import json

json_string = '{"name": "Alice", "age": 30, "is_student": false}'
# Convert JSON to Python dictionary
python_dict = json.loads(json_string)

print(python_dict)

{'name': 'Alice', 'age': 30, 'is_student': False}


In [58]:
import json

python_dict = {"name": "Alice", "age": 30, "is_student": False}

# Convert Python dictionary to JSON
json_string = json.dumps(python_dict)

print(json_string)

{"name": "Alice", "age": 30, "is_student": false}


# Function Calling in LangChain


In [47]:
from langchain.output_parsers.openai_tools import PydanticToolsParser
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Optional


class Article(BaseModel):
    """Identifying key points and contrarian views in an article."""
    points: str = Field(..., description="Key points from the article")
    contrarian_points: Optional[str] = Field(
        None, description="Any contrarian points acknowledged in the article"
    )
    author: Optional[str] = Field(None, description="Author of the article")


_EXTRACTION_TEMPLATE = """Extract and save the relevant entities mentioned \
in the following passage together with their properties.
If a property is not present and is not required in the function parameters,
do not include it in the output."""

# Create a prompt telling the LLM to extract information:
prompt = ChatPromptTemplate.from_messages(
    {("system", _EXTRACTION_TEMPLATE), ("user", "{input}")}
)

model = ChatOpenAI()

pydantic_schemas = [Article]

# Convert Pydantic objects to the appropriate schema:
tools = [convert_to_openai_tool(p) for p in pydantic_schemas]

# Give the model access to these tools:
model = model.bind_tools(tools=tools)

# Create an end to end chain:
chain = prompt | model | PydanticToolsParser(tools=pydantic_schemas)
result = chain.invoke(
    {
        "input": """In the recent article titled 'AI adoption in industry,'
 key points addressed include the growing interest ... However, the
 author, Dr. Jane Smith, ..."""
    }
)

print(result)

[Article(points='the growing interest in AI adoption in various industries', contrarian_points=None, author='Dr. Jane Smith')]


# Extracting Data with LangChain


In [53]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field

# Make sure to use a recent model that supports tools:

model = ChatOpenAI(model="gpt-4o")


class Person(BaseModel):
    name: str = Field(description="The name of the person")
    age: int = Field(description="The age of the person")
    location: str = Field(description="The location of the person")
    hobby: str = Field(description="The hobby of the person")


structured_llm = model.with_structured_output(Person)

result = []
result.append(structured_llm.invoke('''Bob is 25 years old. He lives in New York.
He likes to play basketball.'''))
result.append(structured_llm.invoke('''Sarah is 30 years old. She lives in San
Francisco. She likes to play tennis.'''))
print(result)

[Person(name='Bob', age=25, location='New York', hobby='playing basketball'), Person(name='Sarah', age=30, location='San Francisco', hobby='playing tennis')]


# Query Planner


In [55]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.output_parsers.pydantic import PydanticOutputParser
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
)
from pydantic.v1 import BaseModel, Field
from typing import List


class Query(BaseModel):
    id: int
    question: str
    dependencies: List[int] = Field(
        default_factory=list,
        description="""A list of sub-queries that must be completed before
 this task can be completed.
 Use a sub query when anything is unknown and we might need to ask
 many queries to get an answer.
 Dependencies must only be other queries."""
    )


class QueryPlan(BaseModel):
    query_graph: List[Query]

In [56]:
# Set up a chat model:
model = ChatOpenAI()

# Set up a parser:
parser = PydanticOutputParser(pydantic_object=QueryPlan)
template = """Generate a query plan. This will be used for task execution.
Answer the following query: {query}
Return the following query graph format:
{format_instructions}
"""

system_message_prompt = SystemMessagePromptTemplate.from_template(template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt])

# Create the LCEL chain with the prompt, model, and parser:
chain = chat_prompt | model | parser

result = chain.invoke({
    "query": '''I want to get the results from my database. Then I want to find
out what the average age of my top 10 customers is. Once I have the average
age, I want to send an email to John. Also I just generally want to send a
welcome introduction email to Sarah, regardless of the other tasks.''',
    "format_instructions": parser.get_format_instructions()})

print(result.query_graph)

[Query(id=1, question='I want to get the results from my database.', dependencies=[]), Query(id=2, question='I want to find out what the average age of my top 10 customers is.', dependencies=[1]), Query(id=3, question='I want to send an email to John.', dependencies=[2]), Query(id=4, question='I just generally want to send a welcome introduction email to Sarah.', dependencies=[])]
